In [32]:
import pandas as pd

# 加载所有 parquet 文件
df_trade = pd.read_parquet("/home/ubuntu/crypto_mkt/mkt_signal/scripts/export_data/trade_updates.parquet")
df_order = pd.read_parquet("/home/ubuntu/crypto_mkt/mkt_signal/scripts/export_data/order_updates.parquet")
df_open_sig = pd.read_parquet("/home/ubuntu/crypto_mkt/mkt_signal/scripts/export_data/signals_arb_open.parquet")
df_hedge_sig = pd.read_parquet("/home/ubuntu/crypto_mkt/mkt_signal/scripts/export_data/signals_arb_hedge.parquet")
df_cancel_sig = pd.read_parquet("/home/ubuntu/crypto_mkt/mkt_signal/scripts/export_data/signals_arb_cancel.parquet")
df_close_sig = pd.read_parquet("/home/ubuntu/crypto_mkt/mkt_signal/scripts/export_data/signals_arb_close.parquet")

# 提取 strategy_id
# fn extract_strategy_id(order_id: i64) -> i32 {
#     (order_id >> 32) as i32
# }
df_trade["strategy_id"] = df_trade["client_order_id"].apply(lambda x: (x >> 32) & 0xFFFFFFFF)
df_order["strategy_id"] = df_order["client_order_id"].apply(lambda x: (x >> 32) & 0xFFFFFFFF)

print(f"trade_updates: {len(df_trade)} rows")
print(f"order_updates: {len(df_order)} rows")
print(f"signals_arb_open: {len(df_open_sig)} rows")
print(f"signals_arb_hedge: {len(df_hedge_sig)} rows")
print(f"signals_arb_cancel: {len(df_cancel_sig)} rows")
print(f"signals_arb_close: {len(df_close_sig)} rows")

trade_updates: 4265 rows
order_updates: 10000 rows
signals_arb_open: 10000 rows
signals_arb_hedge: 3429 rows
signals_arb_cancel: 10000 rows
signals_arb_close: 0 rows


In [33]:
# 筛选 LINEA 相关的订单和成交
symbol_filter = "LINEAUSDT"

df_order_linea = df_order[df_order["symbol"] == symbol_filter].copy()
df_trade_linea = df_trade[df_trade["symbol"] == symbol_filter].copy()

print(f"LINEA orders: {len(df_order_linea)} rows")
print(f"LINEA trades: {len(df_trade_linea)} rows")

LINEA orders: 501 rows
LINEA trades: 107 rows


In [40]:
df_order_linea["strategy_id"].head(3)

10    1793496141
11    1793495910
12    1793495686
Name: strategy_id, dtype: int64

In [ ]:
#如果没成交，那么order可以看到挂单和撤单
df_order_linea[df_order_linea["strategy_id"] == 1793496141]

,key,ts_us,event_time,symbol,order_id,client_order_id,client_order_id_str,side,order_type,time_in_force,...,cumulative_filled_quantity,status,raw_status,execution_type,raw_execution_type,trading_venue,average_price,last_executed_price,business_unit,strategy_id
10,00001764176798655809,1764176798655809,1764176798632,LINEAUSDT,196192732,7703007271097204737,NudjJxWif0orxIB3WNX98,BUY,LIMIT,GTC,...,0.0,CANCELED,CANCELED,CANCELED,CANCELED,BinanceMargin,NaN,0.0,None,1793496141
20,00001764176790330001,1764176790330001,1764176790303,LINEAUSDT,196192732,7703007271097204737,7703007271097204737,BUY,LIMIT,GTC,...,0.0,NEW,NEW,NEW,NEW,BinanceMargin,NaN,0.0,None,1793496141


In [ ]:
#create——

In [35]:
df_open_sig[df_open_sig["strategy_id"] == 121123390]

,key,ts_us,strategy_id,create_ts,opening_venue,opening_symbol,opening_bid0,opening_ask0,hedging_venue,hedging_symbol,...,side,order_type,price,price_tick,exp_time,price_offset,hedge_timeout_us,funding_ma,predicted_funding_rate,loan_rate


In [36]:
df_open_sig[df_open_sig["strategy_id"] == 126288409]

,key,ts_us,strategy_id,create_ts,opening_venue,opening_symbol,opening_bid0,opening_ask0,hedging_venue,hedging_symbol,...,side,order_type,price,price_tick,exp_time,price_offset,hedge_timeout_us,funding_ma,predicted_funding_rate,loan_rate


In [37]:
df_trade[df_trade["strategy_id"] == 126288409]

,key,ts_us,event_time,trade_time,symbol,trade_id,order_id,client_order_id,side,price,quantity,commission,commission_asset,is_maker,realized_pnl,trading_venue,cumulative_filled_quantity,order_status,strategy_id


In [38]:
import pandas as pd
df_trade = pd.read_parquet("/home/ubuntu/temp/arb_open_signals.parquet")

FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/temp/arb_open_signals.parquet'